In [ ]:
#%reset -f
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import sympy as sp
import h5py

In [ ]:
import plot_settings as ps

In [ ]:
from PySTFP import PySTFP

In [ ]:
# Load analytical results, which have been calculated using the notebook
# analytical solution.ipynb

analytical_solution = {}

with h5py.File('analytical_solution.h5','r') as f:
    #
    for key in f.keys():
        analytical_solution[key] = f[key][()]

#analytical_dictionary.keys()

In [ ]:
xP = analytical_solution['xP']
P = analytical_solution['P']

a_xP = analytical_solution['a_xP']
D_xP = analytical_solution['D_xP']

a_x0 = analytical_solution['a_x0']
D_x0 = analytical_solution['D_x0']

dt = analytical_solution['dt']
x0 = analytical_solution['x0']


j = a_xP[0] * P[0] \
            - D_xP[1] * P[0] \
            - D_xP[0] * P[1]

Sm_dot_integrand = j/D_xP[0] * ( a_xP[0] - D_xP[1] )

In [ ]:
Sm_dot = np.zeros_like(dt,dtype=float)

for i,current_integrand in enumerate(Sm_dot_integrand):
    Sm_dot[i] = np.trapz(current_integrand,xP)

In [ ]:
Sm_dot_perturbative = []

## Calculate medium entropy production rate from Gaussian propagator

Note that for the Gaussian propagator

\begin{align}
P_{\mathrm{GP}}(\Delta x)
&=
\frac{1}{\sqrt{4 \pi D(x_0) \Delta t}}
\exp\left[
- \frac{ \Delta t}{4D(x_0)} 
\left( \frac{ \Delta x}{\Delta t} - a(x_0)\right)^2
    \right]
\end{align}

it holds that 

\begin{align}
\partial_x P_{\mathrm{GP}}(\Delta x)
&=
-2
 \frac{ 1}{4D(x_0)} 
  \left( \frac{ \Delta x}{\Delta t} - a(x_0)\right) P_{\mathrm{GP}}(\Delta x)
\end{align}

In [ ]:
Sm_dot_Gaussian = np.zeros_like(dt,dtype=float)

for i,current_dt in enumerate(dt):
        #
        P_Gaussian = 1/np.sqrt(4*np.pi*D_x0[0]*current_dt)
        P_Gaussian *= np.exp( - current_dt/(4*D_x0[0]) \
                * ( ( xP - x0)/current_dt - a_x0[0])**2
                )
        #
        dP_Gaussian = - 2/(4*D_x0[0]) \
                * ( ( xP - x0)/current_dt - a_x0[0]) \
                * P_Gaussian
        #
        j_Gaussian = a_xP[0] * P_Gaussian \
                - D_xP[1] * P_Gaussian \
                - D_xP[0] * dP_Gaussian
        #
        integrand_Gaussian = j_Gaussian/D_xP[0] * ( a_xP[0] - D_xP[1] )
        #
        Sm_dot_Gaussian[i] = np.trapz(integrand_Gaussian,xP)


Sm_dot_perturbative.append( Sm_dot_Gaussian )

## Calculate perturbative medium entropy production

In [ ]:
p = PySTFP()

orders_for_perturbative_results = [1,4]

for order in orders_for_perturbative_results:
	#
	current_Sm_dot = p.get_dS_m_dt_from_derivatives(
				D_derivatives=D_x0, # numpy array, 
				# D_derivatives[i] in units of self.D0 / self.L**i
				a_derivatives=a_x0, # numpy array,
				# a_derivatives[i] in units of self.L**(1-i)/self.T
				order=order, # order in epsilon for dimensionless (!) moment
				# note that if dimensionless_position == True, then "order"
				# is the order in physical time
				dimensionless_time=False,
					)
	#
	Sm_dot_perturbative.append( current_Sm_dot(dt) )

## Plot medium entropy production rate

In [ ]:
fig,axes = plt.subplots(3,1,figsize=(6,8))
fig.subplots_adjust(hspace=0.00001,wspace=0.6)

# Subplots:
# axes[0]: Medium entropy production rate
# axes[1]: Relative error for perturbative results
# axes[2]: Running exponent of relative error


# plot vertical line for the breakdown time in all subplots
for i in range(3):
    #
    ps.add_vertical_breakdown_line(ax = axes[i],
                                t_breakdown=analytical_solution['t_breakdown'],
                                )


###########################################
# axes[0]: Medium entropy production rate #
###########################################
ax = axes[0]
ax.set_title('Medium entropy production rate',y=1.03)

# analytical
ax.plot(
    dt,
    Sm_dot,
    color=ps.analytical['color'],
    label=ps.analytical['label'],
    lw=ps.analytical['lw'],
    )

# perturbative
for i, current_Sm_dot in enumerate(Sm_dot_perturbative):
    #
    ax.plot(
        dt,
        current_Sm_dot,
        color=ps.perturbative['color'][i],
        dashes=ps.perturbative['dashes'][i],
        lw=ps.perturbative['lw'][i],
        label=ps.perturbative['label'][i],
    )


####################################################
# axes[1]: Relative error for perturbative results #
####################################################
ax = axes[1]

# plot horizontal line
ax.axhline(ps.horizontal['value'],
            color=ps.horizontal['color'],
            label=ps.horizontal['label'].format(ps.horizontal['value']),
            lw=ps.horizontal['lw'])

for i, current_Sm_dot in enumerate(Sm_dot_perturbative):
    #
    E_rel = np.fabs((current_Sm_dot-Sm_dot)/Sm_dot)
    #
    ax.plot(
        dt,
        E_rel,
        color=ps.perturbative['color'][i],
        dashes=ps.perturbative['dashes'][i],
        lw=ps.perturbative['lw'][i],
        label=ps.perturbative['label'][i],
    )  
    #
    index = np.argmin(np.fabs( E_rel - ps.horizontal['value']) )
    print('{0}: dt = {1:3.3f}, \tE_rel = {2:3.3f}'.format(
                        i,dt[index],ps.horizontal['value'])
                        )


###############################################
# axes[2]: Running exponent of relative error #
###############################################
ax = axes[2]

#
for i, current_Sm_dot in enumerate(Sm_dot_perturbative):
    #
    diff = np.fabs((current_Sm_dot-Sm_dot)/Sm_dot)
    #
    local_exponent = np.log(diff[2:]) - np.log(diff[:-2])
    local_exponent /= np.log(dt[2:]) - np.log(dt[:-2])
    #
    ax.plot(
        dt[1:-1],
        local_exponent,
        color=ps.perturbative['color'][i],
        dashes=ps.perturbative['dashes'][i],
        lw=ps.perturbative['lw'][i],
        label=ps.perturbative['label'][i],
    )  

####################################################
# Set scaling, limits, labels etc for all subplots #
####################################################
xlims = [1e-3,1e0]
ylims = {0:[-0.03,0.22],
        1:[1e-9,1e1],
        2:[0.8,5.5]}

yticks = {0:[0.,0.1,0.2],
        1:[1e-8,1e-6,1e-4,1e-2,1e0],
        2:[1,2,3,4,5]}

ylabels = {0:r'$\dot{S}^{\,\mathrm{m}}\cdot T$',
            1:r'$E_{\mathrm{rel}}$',
            2:r'$\kappa$'}

for i in range(3):
    ax = axes[i]
    #
    ax.set_xscale('log')
    ax.set_xlim(*xlims)
    ax.set_ylim(ylims[i])
    if i == 1:
        ax.set_yscale('log')
    #
    if i < 2:
        ax.set_xticks([])
    ax.set_yticks(yticks[i])
    #
    if i == 2:
        ax.set_xlabel(r'$\Delta t/T$')
    ax.set_ylabel(ylabels[i])
axes[1].set_zorder(-1)
axes[0].legend(loc='lower left',
        bbox_to_anchor=(0.,-0.2),
        framealpha=0.99
        )

#################################################
# Add enumeration (a), (b), (c) to the subplots #
#################################################
enumeration = [ '(a)','(b)','(c)']
for i,e in enumerate(enumeration):
    ax = axes[i]
    #
    xlims = ax.get_xlim()
    dx = xlims[1] - xlims[0]
    ylims = ax.get_ylim()
    dy = ylims[1] - ylims[0]
    #
    ax.text(-0.2,
            .95,
            e,
        horizontalalignment='center',
        verticalalignment='center',
        transform = ax.transAxes,
                fontsize=25)

plt.show()
fig.savefig('Fig4_exact_vs_approximate_medium_entropy_production_rate.pdf',
               bbox_inches='tight')
plt.close(fig)